In [1]:
import requests
import json
from bs4 import BeautifulSoup
import config

# Urls con api news y categorías con Beautiful Soup
El resultado es una lista "seleccionadas" con 5 notas por categoría

In [ ]:
#Contadores y variables
newsapi_key = config.newsAPI_KEY
seleccionadas = []
política, economía, elmundo, deportes, espectáculos, sociedad = 0,0,0,0,0,0
desde= '2018-09-15'
hasta= '2018-09-15'
fuente= 'la-nacion'
pagina = 1

#while hasta que se completen los contadores
while (política < 5) | (economía < 5) | (elmundo < 5) | (deportes < 5) | (espectáculos < 5) | (sociedad < 5):
    print('Contadores de categorías:', end = ' ' )
    print(política, economía, elmundo, deportes, espectáculos, sociedad)
    url = 'https://newsapi.org/v2/everything?sources=' + fuente + '&from=' + desde + '&to=' + hasta + '&page=' + str(pagina) +'&apiKey=' + newsapi_key
    pagina +=1
        
    try: 
        response = requests.get(url)
    except:
        print('no anduvo la api')
        break
    data = json.loads(response.text)
    if data['articles'] == []:
        print('no hay más')
        break

    #for todas la noticias de la pagina
    for i, nota in enumerate(data['articles']):
        response = requests.get(nota['url'])
        html_soup = BeautifulSoup(response.text, 'html.parser')
        for x in html_soup.find_all("strong", {"class": "categoria"}):
            categoria = x.find_all('a')[0].text.replace('\r\n', '').strip()

        #print(categoria, end = ', ')

        if (categoria == 'Política') & (política < 5):
            política +=1
            data['articles'][i]['categoria'] = categoria
            seleccionadas.append(data['articles'][i])
        elif (categoria == 'Economía') & (economía <5):
            economía +=1
            data['articles'][i]['categoria'] = categoria
            seleccionadas.append(data['articles'][i])
        elif (categoria == 'El Mundo') & (elmundo <5):
            elmundo +=1
            data['articles'][i]['categoria'] = categoria
            seleccionadas.append(data['articles'][i])
        elif (categoria == 'Deportes') & (deportes <5):
            deportes +=1
            data['articles'][i]['categoria'] = categoria
            seleccionadas.append(data['articles'][i])
        elif (categoria == 'Espectáculos') & (espectáculos <5):
            espectáculos +=1
            data['articles'][i]['categoria'] = categoria
            seleccionadas.append(data['articles'][i])
        elif (categoria == 'Sociedad') & (sociedad <5):
            sociedad +=1
            data['articles'][i]['categoria'] = categoria
            seleccionadas.append(data['articles'][i])
        
print('Éxito')
#que pasa si con todas las paginas que se recuperan de la api no se llega a llenar la cuota? puede pasar?


In [ ]:
len(seleccionadas)

# Selenium para conseguir comments

In [2]:
#Acá habría que hacerlo para toda la lista, pero pruebo sólo con la primera
#url = seleccionadas[0]['url']
url = 'https://www.lanacion.com.ar/2181492-hugo-moyano-pedido-detencion-hijo'
url = 'https://www.lanacion.com.ar/2183250-piden-juicio-oral-daniel-scioli-delitos-corrupcion'

In [260]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

windows = '/Users/p_bag/Downloads/chromedriver_win32/chromedriver.exe'
linux = '/home/pablo/Descargas/chromedriver'
 
driver = webdriver.Chrome(linux)
driver.get(url)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

try:
    myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'fyre-stream-more-container')))
    print ("Hay boton")
    button = True
except TimeoutException:
    print ("No hay boton")
    button = False
#Que pasa si la nota esta cerrada a comentarios

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=70.0.3538.67)
  (Driver info: chromedriver=2.42.591071 (0b695ff80972cc1a65a5cd643186d2ae582cd4ac),platform=Linux 4.15.0-34-generic x86_64)


In [ ]:
while button:
    try:
        driver.find_element_by_class_name('fyre-stream-more-container').click()
    except: button = False

In [ ]:
#todos los comentarios con me gusta, horario y autores,etc
todo = driver.find_element_by_class_name('fyre-stream-content')

In [ ]:
#version con conteo de respuestas

In [ ]:
a= todo.find_elements_by_class_name('fyre-comment-article')

In [ ]:
#cantidad de respuestas
def cantidad_respuestas(i):
    autor = '@' + a[i].find_elements_by_class_name("fyre-comment-username")[0].text
    respuesta = 0
    for x in a[i].find_elements_by_class_name('fyre-comment'):
        if (autor in x.text):
            respuesta +=1 
    return respuesta

In [ ]:
#cada a ver cuantos y guardar texto [0], y los otros?
import numpy as np
def get_comment(indice):
    comentario = {}
    #print(indice)
    try:
        comentario['texto'] = a[indice].find_elements_by_class_name("fyre-comment")[0].text
        comentario['usuario'] = a[indice].find_elements_by_class_name("fyre-comment-username")[0].text
        comentario['likes'] = a[indice].find_elements_by_class_name("fyre-comment-like-count")[0].text
        comentario['fecha'] = a[indice].find_elements_by_class_name("fyre-comment-date")[0].text
        comentario['respuestas'] = cantidad_respuestas(indice)
    except:
        print(indice)
        #comentario['texto'] = np.nan
        #comentario['usuario'] = np.nan
        #comentario['likes'] = np.nan
        #comentario['fecha'] = np.nan
        #comentario['respuestas'] = np.nan
    return comentario

In [ ]:
def get_comments(lista):
    return [get_comment(x) for x in lista]

In [ ]:
get_comments(np.arange(0,4))

In [ ]:
#%%time
#comentarios = [get_comment(x) for x in range(len(a))]

In [247]:
len(a)

162

In [251]:
listas = np.array_split(np.arange(0, len(a)), 4)

In [252]:
from joblib import Parallel, delayed

In [254]:
%%time
results = Parallel(n_jobs=-1)(delayed(get_comment)(rangos) for rangos in listas)

PicklingError: Could not pickle the task to send it to the workers.

In [228]:
import pandas as pd
df1 = pd.DataFrame(comentarios)

In [229]:
df1 = df1.dropna().reset_index(drop = True)
df1.loc[:, 'respuestas'] = df1.respuestas.astype(int)

In [198]:
#versión sin conteo de respuestas

In [199]:
%%time
textos = todo.find_elements_by_class_name('fyre-comment')
users = todo.find_elements_by_class_name("fyre-comment-username")
fechas = todo.find_elements_by_class_name("fyre-comment-date")
likes = todo.find_elements_by_class_name("fyre-comment-like-count")

CPU times: user 12 ms, sys: 16 µs, total: 12 ms
Wall time: 353 ms


In [200]:
%%time
usuarios =[x.text for x in users]
textos = [x.text for x in textos]
fechas = [x.text for x in fechas]
likes = [x.text for x in likes]

CPU times: user 820 ms, sys: 108 ms, total: 928 ms
Wall time: 31.6 s


In [201]:
import pandas  as pd
df = pd.DataFrame({'usuario': usuarios,
                     'comment': texto,
                   'fecha': fechas,
                   'likes': likes
                  })

In [202]:
df.shape

(158, 4)

In [235]:
driver.quit()

In [12]:
#df.to_csv('./data/hugo-moyano-pedido-detencion-hijo.csv')

In [233]:
df

,usuario,comment,fecha,likes
0,juanjorgesalzman,"Lopez,Jaime,Baez,Zannini,Bousuo,De Vido,Manzan...",16:05 19/10/2018,
1,Proteo1,Parece que no se usan los trolls para defender...,15:38 19/10/2018,
2,mrojo3,este debería estar presi hace mucho y confisca...,14:15 19/10/2018,
3,coco6344,¡Scioli no volvés más!,14:08 19/10/2018,
4,mmurgia,Tiene también la fábrica de Felpudos jajajajaj...,13:44 19/10/2018,1
5,Vitolino,fue jefe de una asociación ilícita. lo que es...,13:40 19/10/2018,2
6,juanjorgesalzman,Que raro Scioli sospechado de corrupcion….,13:39 19/10/2018,3
7,rarregui4,Este impresentable.en otro pais.ya hubiera cum...,13:34 19/10/2018,4
8,EricGaynor,Scioli hizo quebrar a todas las empresas que c...,13:15 19/10/2018,3
9,Saccardi,...el motonauta quiso circular por la ancha av...,13:11 19/10/2018,1


In [234]:
df1

,fecha,likes,respuestas,texto,usuario
0,16:05 19/10/2018,,0,"Lopez,Jaime,Baez,Zannini,Bousuo,De Vido,Manzan...",juanjorgesalzman
1,15:38 19/10/2018,,0,Parece que no se usan los trolls para defender...,Proteo1
2,14:15 19/10/2018,,0,este debería estar presi hace mucho y confisca...,mrojo3
3,14:08 19/10/2018,,0,¡Scioli no volvés más!,coco6344
4,13:44 19/10/2018,1,0,Tiene también la fábrica de Felpudos jajajajaj...,mmurgia
5,13:40 19/10/2018,2,0,fue jefe de una asociación ilícita. lo que es...,Vitolino
6,13:39 19/10/2018,3,0,Que raro Scioli sospechado de corrupcion….,juanjorgesalzman
7,13:34 19/10/2018,4,0,Este impresentable.en otro pais.ya hubiera cum...,rarregui4
8,13:15 19/10/2018,3,0,Scioli hizo quebrar a todas las empresas que c...,EricGaynor
9,13:11 19/10/2018,1,0,...el motonauta quiso circular por la ancha av...,Saccardi
